In [17]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['OMP_NUM_THREADS'] = '1'

In [24]:
import os
import json
import joblib

import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder
import h5py

from lightfm import LightFM

In [19]:
!mkdir models

mkdir: cannot create directory ‘models’: File exists


In [37]:
def load_hdf_to_df(file_path, dataset_name):
    with h5py.File(file_path, 'r') as f:
        # print(f['abc'])
        data = f[dataset_name]
        df = pd.DataFrame(f)
    return df

# Load all datasets using the function
df_tracks = load_hdf_to_df('df_data/df_tracks.hdf', 'abc')
# df_playlists = load_hdf_to_df('df_data/df_playlists.hdf', 'df_playlists')
# df_playlists_info = load_hdf_to_df('df_data/df_playlists_info.hdf', 'df_playlists_info')
# df_playlists_test = load_hdf_to_df('df_data/df_playlists_test.hdf', 'df_playlists_test')
# df_playlists_test_info = load_hdf_to_df('df_data/df_playlists_test_info.hdf', 'df_playlists_test_info')

display(df_tracks)

,0
0,abc


In [ ]:
train = pd.read_hdf('df_data/train.hdf')
val = pd.read_hdf('df_data/val1.hdf')
val1_pids = joblib.load('df_data/val1_pids.pkl')

In [6]:
user_seen = train.groupby('pid').tid.apply(set).to_dict()
val_tracks = val.groupby('pid').tid.apply(set).to_dict()

In [7]:
config = {
    'num_playlists': df_playlists_test_info.pid.max() + 1,
    'num_tracks': df_tracks.tid.max() + 1,
}

In [8]:
X_train = sp.coo_matrix(
    (np.ones(len(train)), (train.pid, train.tid)),
    shape=(config['num_playlists'], config['num_tracks'])
)

In [16]:
config['model_path'] = 'models/lightfm_model.pkl'

print(val)

Empty DataFrame
Columns: [Unnamed: 0.1, Unnamed: 0, pid, tid, pos]
Index: []


In [12]:
model = LightFM(no_components=200, loss='warp', learning_rate=0.02, max_sampled=400, random_state=1, user_alpha=1e-05)

best_score = 0
for i in range(60):
    
    # model.fit_partial(X_train, epochs=5, num_threads=8)

    # model.batch_setup(
    #     item_chunks={0: np.arange(config['num_tracks'])},
    #     n_process=50, 
    # )
    # res = model.batch_predict(chunk_id=0, user_ids=val1_pids, top_k=600)
    # model.batch_cleanup()

    model.fit(X_train, epochs=5, num_threads=6)
    res = model.predict(user_ids=val1_pids, item_ids=np.arange(config['num_tracks']))
    
    score = []
    for pid in val1_pids:
        tracks_t = val_tracks[pid]
        tracks = [i for i in res[pid][0] if i not in user_seen.get(pid, set())][:len(tracks_t)]
        guess = np.sum([i in tracks_t for i in tracks])
        score.append(guess / len(tracks_t))
    
    score = np.mean(score)
    print(score)
    if score > best_score:
        joblib.dump(model, open(config['model_path'], 'wb'))
        best_score = score

ValueError: Expected the number of user IDs (0) to equal the number of item IDs (57884)